In [3]:
import requests
from requests.models import Response
from requests.structures import CaseInsensitiveDict


In [4]:
response: Response = requests.get('https://simple-books-api.glitch.me/status')
print(response)
print(response.status_code)
print(response.json())
print(response.headers)

<Response [200]>
200
{'status': 'OK'}
{'Date': 'Sun, 24 Dec 2023 16:50:07 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '15', 'Connection': 'keep-alive', 'x-powered-by': 'Express', 'etag': 'W/"f-v/Y1JusChTxrQUzPtNAKycooOTA"'}


## Get List of Books

In [5]:
def get_all_books():
    try:
        response = requests.get('https://simple-books-api.glitch.me/books')
        response.raise_for_status() # This line will raise an HTTPError if the response was not successful
        res = response.json()
        return res
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
get_all_books()

[{'id': 1, 'name': 'The Russian', 'type': 'fiction', 'available': True},
 {'id': 2, 'name': 'Just as I Am', 'type': 'non-fiction', 'available': False},
 {'id': 3, 'name': 'The Vanishing Half', 'type': 'fiction', 'available': True},
 {'id': 4,
  'name': 'The Midnight Library',
  'type': 'fiction',
  'available': True},
 {'id': 5, 'name': 'Untamed', 'type': 'non-fiction', 'available': True},
 {'id': 6,
  'name': 'Viscount Who Loved Me',
  'type': 'fiction',
  'available': True}]

## Get a Single Book

In [23]:
response: Response = requests.get('https://simple-books-api.glitch.me/books/1')
status_code : int = response.status_code
print(status_code)
json_obj : dict = response.json()
pprint.pprint(json_obj)


200
{'author': 'James Patterson and James O. Born',
 'available': True,
 'current-stock': 12,
 'id': 1,
 'isbn': '1780899475',
 'name': 'The Russian',
 'price': 12.98,
 'type': 'fiction'}


## Get Book ID by Name

In [6]:
def get_book_id(name):
    response = requests.get('https://simple-books-api.glitch.me/books')
    res = response.json()

    for book in res:
        if book['name'] == name:
            return book['id']



## Get Price BY Name

In [7]:
def getprice(name):
    bookid = get_book_id(name)
    response = requests.get(f"https://simple-books-api.glitch.me/books/{bookid}")
    res = response.json()
    return res['price']

getprice("The Russian")

12.98

## Get Single Book By Name

In [8]:
def get_single_book(name):
    bookid = get_book_id(name)
    response = requests.get(f"https://simple-books-api.glitch.me/books/{bookid}")
    res = response.json()
    return res

get_single_book("The Russian")

{'id': 1,
 'name': 'The Russian',
 'author': 'James Patterson and James O. Born',
 'isbn': '1780899475',
 'type': 'fiction',
 'price': 12.98,
 'current-stock': 12,
 'available': True}

In [9]:
from openai import OpenAI
import json
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv(find_dotenv()) # read local .env file

client : OpenAI = OpenAI()

In [12]:
from openai.types.chat.chat_completion import ChatCompletionMessage, ChatCompletion
# from openai.types.chat.chat_completion import ChatCompletionMessageParam, ChatCompletionMessageParam

def run_conversation(main_request: str)->str:
    # Step 1: send the conversation and available functions to the model
    messages= [
      {"role": "assistant", "content": "You are only a bookstore assistant. You should only tell users about the names, prices, availability, and other details of books. If any other question arises, you should refuse to answer it."},
      {"role": "user", "content": main_request}
    ]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_all_books",
                "description": "Get the names of all book in our list",
            },
        },
        {
            "type": "function",
            "function": {
                "name": "getprice",
                "description": "Get the price of single book",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "name": {
                            "type": "string",
                            "description": "name of book, e.g. The Russian",
                        }},
                    "required": ["name"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_single_book",
                "description": "Get the name of single book",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "name": {
                            "type": "string",
                            "description": "name of book, e.g. The Russian",
                        }},
                    "required": ["name"],
                },
            },
        }
    ]

    # First Request
    response: ChatCompletion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message: ChatCompletionMessage = response.choices[0].message
    display("* First Response: ", dict(response_message))

    tool_calls = response_message.tool_calls
    display("* First Reponse Tool Calls: ", list(tool_calls))

    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_all_books": get_all_books,
            "getprice": getprice,
            "get_single_book": get_single_book,
            
        }  # only one function in this example, but you can have multiple
        
        messages.append(response_message)  # extend conversation with assistant's reply
        
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            if function_name in available_functions:
                    function_to_call = available_functions[function_name]
                    function_response  = function_to_call(**function_args)
                    print("i am here")    
                        
                    messages.append({
                                "tool_call_id": tool_call.id,
                                "role": "tool",
                                "name": function_name,
                                "content": function_response,
                            })
                          # extend conversation with function response
        display("* Second Request Messages: ", list(messages))
        second_response: ChatCompletion = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        print("* Second Response: ", dict(second_response))
        return second_response.choices[0].message.content
        # return second_response.choices[0].message.content

In [13]:
run_conversation("whichs book do you have")


'* First Response: '

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [ChatCompletionMessageToolCall(id='call_pWEU68t1bgc9d91AHcKDT8Ea', function=Function(arguments='{}', name='get_all_books'), type='function')]}

'* First Reponse Tool Calls: '

[ChatCompletionMessageToolCall(id='call_pWEU68t1bgc9d91AHcKDT8Ea', function=Function(arguments='{}', name='get_all_books'), type='function')]

i am here


'* Second Request Messages: '

[{'role': 'assistant',
  'content': 'You are only a bookstore assistant. You should only tell users about the names, prices, availability, and other details of books. If any other question arises, you should refuse to answer it.'},
 {'role': 'user', 'content': 'whichs book do you have'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_pWEU68t1bgc9d91AHcKDT8Ea', function=Function(arguments='{}', name='get_all_books'), type='function')]),
 {'tool_call_id': 'call_pWEU68t1bgc9d91AHcKDT8Ea',
  'role': 'tool',
  'name': 'get_all_books',
  'content': [{'id': 1,
    'name': 'The Russian',
    'type': 'fiction',
    'available': True},
   {'id': 2,
    'name': 'Just as I Am',
    'type': 'non-fiction',
    'available': False},
   {'id': 3,
    'name': 'The Vanishing Half',
    'type': 'fiction',
    'available': True},
   {'id': 4,
    'name': 'The Midnight Library',
    'type': 'fiction',
    'available': True},
   {

BadRequestError: Error code: 400 - {'error': {'message': "'$.messages[3].content' is invalid. Please check the API reference: https://platform.openai.com/docs/api-reference.", 'type': 'invalid_request_error', 'param': None, 'code': None}}